In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
import datetime
import import_ipynb
from tensorboard.plugins.hparams import api as hp
from dataloader import get_LFW_X_df, get_y_df, data_prep

2023-11-18 15:37:33.762143: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-18 15:37:33.791601: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-18 15:37:33.792176: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-18 15:37:34.439087: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


importing Jupyter notebook from dataloader.ipynb
Tensorflow version: 2.12.0


In [3]:
rm -rf ./logs/

In [4]:
csv_filepath = "../../data/csv/plant_data.csv"

In [5]:
X = get_LFW_X_df(csv_filepath)
y = get_y_df(csv_filepath, 'LFW_g')

In [6]:
HP_NUM_UNITS = hp.HParam("num_units", hp.Discrete([16, 32, 64, 128]))
HP_DROPOUT = hp.HParam("dropout", hp.RealInterval(0.1, 0,2))
HP_OPTIMIZER = hp.HParam("optimizer", hp.Discrete(['adam', 'sgd']))
HP_ACTIVATION = hp.HParam("activation". hp.Discrete(['linear', 'relu', 'sigmoid']))

METRIC_ACCURACY = 'val_accuracy'

In [ ]:
with tf.summary.create_file_writer('../../logs/hyperparam_tuning/repression/LFW').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER, HP_ACTIVATION],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name="Validation Accuracy")]
    )

In [7]:
X_train, X_test, y_train, y_test = data_prep(X, y)

In [8]:
def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], input_shape = (11,), hparams[HP_ACTIVATION], name='layers_input'),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], hparams[HP_ACTIVATION], name='layers_dense2'),
        tf.keras.layers.Dropout(HP_DROPOUT),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1, name='layers_dense3')
    ])

In [9]:
model = create_model()
model.compile(optimizer=hparams[HP_OPTIMIZER],
              loss='mae',
            )

2023-11-18 15:37:35.684681: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-18 15:37:35.702053: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [10]:
log_dir = "../../logs/regression/LFW/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [11]:
model.fit(X_train,
          y_train,
          epochs=100,
          validation_split=0.2,
          callbacks=[
            tf.keras.callbacks.TensorBoard(log_dir=log_dir),
            hp.KerasCallback(log_dir, hparams)
            ]
          )

Epoch 1/500


11/11 [==============================] - 1s 19ms/step - loss: 62.5226 - val_loss: 59.2570
Epoch 2/500
11/11 [==============================] - 0s 7ms/step - loss: 62.3624 - val_loss: 59.1711
Epoch 3/500
11/11 [==============================] - 0s 7ms/step - loss: 62.2375 - val_loss: 59.0539
Epoch 4/500
11/11 [==============================] - 0s 7ms/step - loss: 62.0983 - val_loss: 58.8989
Epoch 5/500
11/11 [==============================] - 0s 7ms/step - loss: 61.9374 - val_loss: 58.7028
Epoch 6/500
11/11 [==============================] - 0s 7ms/step - loss: 61.7494 - val_loss: 58.4851
Epoch 7/500
11/11 [==============================] - 0s 7ms/step - loss: 61.5301 - val_loss: 58.2279
Epoch 8/500
11/11 [==============================] - 0s 7ms/step - loss: 61.2812 - val_loss: 57.9611
Epoch 9/500
11/11 [==============================] - 0s 7ms/step - loss: 60.9907 - val_loss: 57.6306
Epoch 10/500
11/11 [==============================] - 0s 7ms/step - loss: 60.6635 - val_loss: 57.2366
